# TESTE1 – Diagnóstico MinIO
Notebook para verificar o conflito na porta 9000 e a conectividade do ambiente `ds-base` com o servidor MinIO em Docker.

In [9]:
!ss -ltnp | grep 9000 || true

LISTEN 0      4096         0.0.0.0:9000       0.0.0.0:*                                      
LISTEN 0      4096            [::]:9000          [::]:*                                      


In [10]:
from pathlib import Path
from typing import Dict
import json
import os

In [11]:
ROOT = Path.cwd().resolve().parents[1]
ENV_FILE = ROOT / 'infra' / 'env' / 'minio_credentials.env'
ENV_FILE

PosixPath('/home/wilson/Maringa/fase_1_diagnostico/infra/env/minio_credentials.env')

In [12]:
def load_env(path: Path) -> Dict[str, str]:
    values: Dict[str, str] = {}
    for raw_line in path.read_text().splitlines():
        line = raw_line.strip()
        if not line or line.startswith('#') or '=' not in line:
            continue
        key, value = line.split('=', 1)
        values[key.strip()] = value.strip()
    return values

creds = load_env(ENV_FILE)
creds

{'MINIO_ROOT_USER': 'WRM',
 'MINIO_ROOT_PASSWORD': 'faty7098',
 'MINIO_SERVER_PORT': '9000',
 'MINIO_SERVER_CONSOLE_PORT': '9001'}

In [13]:
%pip install -q minio

Note: you may need to restart the kernel to use updated packages.


In [14]:
from minio import Minio

client = Minio(
    endpoint=f"localhost:{creds.get('MINIO_SERVER_PORT', '9000')}",
    access_key=creds['MINIO_ROOT_USER'],
    secret_key=creds['MINIO_ROOT_PASSWORD'],
    secure=False,
 )

client.list_buckets()

[Bucket(name='maringa-derived', creation_date=datetime.datetime(2025, 11, 15, 16, 23, 2, 909000, tzinfo=datetime.timezone.utc)),
 Bucket(name='maringa-manifests', creation_date=datetime.datetime(2025, 11, 15, 16, 23, 2, 922000, tzinfo=datetime.timezone.utc)),
 Bucket(name='maringa-raw', creation_date=datetime.datetime(2025, 11, 15, 16, 23, 2, 881000, tzinfo=datetime.timezone.utc)),
 Bucket(name='maringa-refined', creation_date=datetime.datetime(2025, 11, 15, 16, 23, 2, 898000, tzinfo=datetime.timezone.utc))]

In [15]:
TARGET_BUCKETS = [
    'maringa-raw',
    'maringa-refined',
    'maringa-derived',
    'maringa-manifests',
]

creation_status = {}
for name in TARGET_BUCKETS:
    if client.bucket_exists(name):
        creation_status[name] = 'já existia'
        continue
    client.make_bucket(name)
    creation_status[name] = 'criado'

creation_status

{'maringa-raw': 'já existia',
 'maringa-refined': 'já existia',
 'maringa-derived': 'já existia',
 'maringa-manifests': 'já existia'}

## Observações
- Porta 9000 ocupada pelo `docker-proxy` do container MinIO, como esperado.
- Pacote `minio` instalado via `%pip` dentro do kernel antes da importação.
- Buckets `maringa-*` criados com sucesso usando o cliente local `localhost:9000`.
